In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models

#from tqdm import tqdm
from tqdm import tqdm_notebook as tqdm
from time import sleep

#import models.VGG 

# DataLoader(CIFAR10)

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=100,
                                          shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=100,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


## Model

In [3]:
model = models.vgg11_bn(pretrained=True) 
model.classifier[0] = nn.Linear(512, 4096)
model.classifier[6] = nn.Linear(model.classifier[6].in_features, len(classes))
#model = models.VGG.VGG(len(classes))
model.cuda()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace)
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace)
    (14): MaxPool2d(kernel_size=2, stride=

# Optimizer

In [4]:
#optimizer = optim.SGD(model.parameters(),
#                    lr=0.01,
#                    momentum=0.9,
#                    weight_decay=0,
#                    nesterov=True)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Loss

In [5]:
criterion = nn.CrossEntropyLoss()

# Train 1 epoch function 

In [6]:
def train_loop(model, loader, criterion, optimizer):
    model.train()
    bar = tqdm(total=len(loader), leave=False)
    total_loss, total_acc, total_num = 0, 0, 0
    for idx, feed in enumerate(loader):
        # Prepare data
        inputs, labels = feed
        inputs, labels = inputs.cuda(), labels.cuda()
        # Foward
        outputs = model(inputs)
        # Calcurate Loss
        loss = criterion(outputs, labels)
        # initialize gradient
        optimizer.zero_grad()
        # Backward
        loss.backward()
        # Update Params
        optimizer.step()
        # Update bar
        ## Accuracy
        pred = outputs.data.max(1, keepdim=True)[1]
        acc = pred.eq(labels.data.view_as(pred)).sum()
        ## Calcurate Score
        total_loss += loss.item() * labels.size(0)
        total_acc += acc.item()
        total_num += labels.size(0)
        bar.set_description("Loss: {:.4f}, Accuracy: {:.2f}".format(
            total_loss / total_num, total_acc / total_num * 100), refresh=True)
        bar.update()
    bar.close()
    return total_loss / total_num, total_acc / total_num * 100

# Validation function

In [7]:
def valid_loop(model, loader, criterion):
    model.eval()
    total_loss, total_acc, total_num = 0, 0, 0
    bar = tqdm(test_loader, total=len(test_loader), leave=False)
    for i, feed in enumerate(loader):
        with torch.no_grad():
            # Prepare data
            inputs, labels = feed
            inputs = inputs.cuda()
            labels = labels.cuda()
            # Foward
            outputs = model(inputs)
            # Calcurate Loss
            loss = criterion(outputs, labels)
            # Update bar
            ## Accuracy
            pred = outputs.data.max(1, keepdim=True)[1]
            acc = pred.eq(labels.data.view_as(pred)).sum()
            ## Calcurate Score
            total_loss += loss.item() * labels.size(0)
            total_acc += acc.item()
            total_num += labels.size(0)

            bar.set_description("Loss: {:.4f}, Accuracy: {:.2f}".format(
                total_loss / total_num, total_acc / total_num * 100), refresh=True)
            bar.update()
    bar.close()
    return total_loss / total_num, total_acc / total_num * 100

In [8]:
max_epochs = 5
for e in range(max_epochs):
    train_loss, train_acc = train_loop(model, train_loader, criterion, optimizer)
    test_loss, test_acc = valid_loop(model, test_loader, criterion)
    print('Epoch: {}, Train Loss: {:.4f}, Train Accuracy: {:.2f}, Test Loss: {:.4f}, Test Accuracy: {:.2f}'.format(
    e + 1, train_loss, train_acc, test_loss, test_acc))


Epoch: 1, Train Loss: 0.7759, Train Accuracy: 74.93, Test Loss: 0.7103, Test Accuracy: 77.33


Epoch: 2, Train Loss: 0.4703, Train Accuracy: 85.11, Test Loss: 0.5580, Test Accuracy: 82.32


Epoch: 3, Train Loss: 0.3220, Train Accuracy: 89.75, Test Loss: 0.5452, Test Accuracy: 83.19


Epoch: 4, Train Loss: 0.2412, Train Accuracy: 92.45, Test Loss: 0.5159, Test Accuracy: 84.67


Epoch: 5, Train Loss: 0.1834, Train Accuracy: 94.31, Test Loss: 0.6018, Test Accuracy: 83.96


In [9]:
torch.save(model.state_dict(), 'params/model.pth')